# EOPF S3 OLCI L1 Product Data Structure Proposal


In [4]:
import os
import xarray as xr
import glob
import re
from EOProductDataStructure import EOProductBuilder, EOVariableBuilder, EOGroupBuilder
from lxml import etree

ModuleNotFoundError: No module named 'rasterio'

In [5]:
variable_chunks = { 'B01': 192, 
                    'B02': 1024, 'B03': 1024, 'B04': 1024, 
                    'B05': 640, 'B06': 640, 'B07': 640, 'B08': 640, 'B8A': 640, 
                    'B09': 192, 'B10': 192, 
                    'B11': 640, 'B12': 640, 
                    'TCI': 256 }

def get_jp2_ds(path_to_product, glob_patterns, var_pattern, resolution):
    variables = {}
    coordinates = {}
    attributes = {}
    for glob_pattern in glob_patterns:
        files = glob.glob(path_to_product + '/' + glob_pattern)
        for file in files:
            var = re.match(var_pattern, file[file.rfind('/')+1:]).group(1)
            chunks = variable_chunks[var]
            ds1 = xr.open_dataset(file, chunks=chunks, engine='rasterio', mask_and_scale=False)
            if var == 'TCI':
                variables['red'] = ds1.get('band_data')[0].drop('band')
                variables['green'] = ds1.get('band_data')[1].drop('band')
                variables['blue'] = ds1.get('band_data')[2].drop('band')
            else:
                variables[var] = ds1.get('band_data')[0].drop('band')
            for attr in ds1.attrs:
                if attr not in attributes:
                    attributes[attr] = ds1.attrs[attr]
    ds = xr.Dataset(data_vars=variables, coords=coordinates, attrs=attributes).rename({'x': 'x_'+resolution, 'y': 'y_'+resolution}).drop(['spatial_ref', 'x_'+resolution, 'y_'+resolution])
    return ds

def get_coord_ds(path_to_product, glob_patterns, resolutions):
    variables = {}
    coordinates = {}
    attributes = {}
    for glob_pattern, resolution in zip(glob_patterns, resolutions):
        files = glob.glob(path_to_product + '/' + glob_pattern)
        for file in files:
            ds1 = xr.open_dataset(file, engine='rasterio', mask_and_scale=False).rename({'x': 'x_'+resolution, 'y': 'y_'+resolution}) 
            variables['x_' + resolution] = ds1['x_' + resolution]
            variables['y_' + resolution] = ds1['y_' + resolution]
            if 'spatial_ref' in ds1 and 'spatial_ref' not in variables:
                variables['spatial_ref'] = ds1['spatial_ref']
            for attr in ds1.attrs:
                if attr not in attributes:
                    attributes[attr] = ds1.attrs[attr]
    ds = xr.Dataset(data_vars=variables, coords=coordinates, attrs=attributes)
    return ds

In [6]:

band_names = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B10', 'B11', 'B12']

def get_values(dom, xpath):
    list = dom.xpath(xpath, namespaces={'n1': 'https://psd-14.sentinel2.eo.esa.int/PSD/S2_PDI_Level-1C_Tile_Metadata.xsd'})
    array = [[float(i) for i in x.text.split()] for x in list]
    da = xr.DataArray(array, dims=['y_tiepoints', 'x_tiepoints'])
    return da

def get_shape(dom, xpath):
    list = dom.xpath(xpath, namespaces={'n1': 'https://psd-14.sentinel2.eo.esa.int/PSD/S2_PDI_Level-1C_Tile_Metadata.xsd'})
    return [len(list), len(list[0].text.split())]

def parse_xml(path_to_product, glob_pattern):
    path = glob.glob(path_to_product + '/' + glob_pattern)[0]
    dom = etree.parse(path)
    return dom

def get_angles_ds(path_to_product, glob_pattern):
    dom = parse_xml(path_to_product, glob_pattern)
    sza = get_values(dom, 'n1:Geometric_Info/Tile_Angles/Sun_Angles_Grid/Zenith/Values_List/VALUES')
    saa = get_values(dom, 'n1:Geometric_Info/Tile_Angles/Sun_Angles_Grid/Azimuth/Values_List/VALUES')
    bands = {'sza': sza, 'saa': saa}
    for band_id in range(13):
        for detector_id in range(1,7):
            vza = get_values(dom, 'n1:Geometric_Info/Tile_Angles/Viewing_Incidence_Angles_Grids[@bandId="{}" and @detectorId="{}"]/Zenith/Values_List/VALUES'
                             .format(band_id, detector_id))
            vaa = get_values(dom, 'n1:Geometric_Info/Tile_Angles/Viewing_Incidence_Angles_Grids[@bandId="{}" and @detectorId="{}"]/Azimuth/Values_List/VALUES'
                             .format(band_id, detector_id))
            bands['vza_{}_{}'.format(band_names[band_id], detector_id)] = vza
            bands['vaa_{}_{}'.format(band_names[band_id], detector_id)] = vaa
    ds = xr.Dataset(bands)
    return ds    

def get_tiepoints_ds(path_to_product, glob_pattern):
    dom = parse_xml(path_to_product, glob_pattern)
    shape_y_x = get_shape(dom, 'n1:Geometric_Info/Tile_Angles/Sun_Angles_Grid/Zenith/Values_List/VALUES')
    ymax = float(dom.xpath('n1:Geometric_Info/Tile_Geocoding/Geoposition[@resolution="10"]/ULY',
                           namespaces={'n1': 'https://psd-14.sentinel2.eo.esa.int/PSD/S2_PDI_Level-1C_Tile_Metadata.xsd'})[0].text)
    xmin = float(dom.xpath('n1:Geometric_Info/Tile_Geocoding/Geoposition[@resolution="10"]/ULX',
                           namespaces={'n1': 'https://psd-14.sentinel2.eo.esa.int/PSD/S2_PDI_Level-1C_Tile_Metadata.xsd'})[0].text)
    ystep = float(dom.xpath('n1:Geometric_Info/Tile_Angles/Sun_Angles_Grid/Zenith/ROW_STEP', 
                            namespaces={'n1': 'https://psd-14.sentinel2.eo.esa.int/PSD/S2_PDI_Level-1C_Tile_Metadata.xsd'})[0].text)
    xstep = float(dom.xpath('n1:Geometric_Info/Tile_Angles/Sun_Angles_Grid/Zenith/COL_STEP', 
                            namespaces={'n1': 'https://psd-14.sentinel2.eo.esa.int/PSD/S2_PDI_Level-1C_Tile_Metadata.xsd'})[0].text)
    y = [ymax - i * ystep - ystep / 2 for i in range(shape_y_x[0])]
    x = [xmin + i * xstep + xstep / 2 for i in range(shape_y_x[1])]
    ds = xr.Dataset({'y_tiepoints': y, 'x_tiepoints': x})
    return ds
    

In [7]:
path_to_product = glob.glob("data/S2?_MSIL1C*.SAFE")[0]

# Groups definition
groups = {}
groups['coordinates'] = get_coord_ds(path_to_product, ["GRANULE/*/IMG_DATA/*_%s.jp2" % r for r in ['B02','B05','B01']], ['10m', '20m', '60m'])  # extensional coordinates, metric and geographic
groups['tiepoints'] = get_tiepoints_ds(path_to_product, "GRANULE/*/MTD_TL.xml")
#groups['crs'] = get_crs_ds(path_to_product, [""])  # utm zone, geographic footprint, metric corners, metric resolutions, parameters to feed proj
groups['measurements_10m'] = get_jp2_ds(path_to_product,["GRANULE/*/IMG_DATA/*_%s.jp2" % r for r in ['B02','B03','B04','B08']], '.*_(...).jp2', '10m')
groups['measurements_20m'] = get_jp2_ds(path_to_product,["GRANULE/*/IMG_DATA/*_%s.jp2" % r for r in ['B05','B06','B07','B8A','B11','B12']], '.*_(...).jp2', '20m')
groups['measurements_60m'] = get_jp2_ds(path_to_product,["GRANULE/*/IMG_DATA/*_%s.jp2" % r for r in ['B01','B09','B10']], '.*_(...).jp2', '60m')
groups['quicklook_tci'] = get_jp2_ds(path_to_product,["GRANULE/*/IMG_DATA/*_%s.jp2" % r for r in ['TCI']], '.*_(...).jp2', '10m')
groups['geometry'] = get_angles_ds(path_to_product,"GRANULE/*/MTD_TL.xml")  # angles on tiepoint raster
#groups['instrument'] = get_xml_ds(path_to_product,["MTD_MSIL1C.xml"])  # band characteristics, gains

#groups['meteo'] = get_ds(path_to_product,["tie_meteo"])

# Create a new EOProduct instance
product_name = os.path.basename("S2_MSIL1C")
product = EOProductBuilder("S2_MSIL1C")
# do the same work as before 
product.metadatas = ["MTD_MSIL1C.xml"]

# ==================== Product groups setting ========================

for group_name, ds in groups.items():
    group = EOGroupBuilder(group_name)
    group.attrs["description"] = f"{group_name} Data Group"
    group.dims = ds.dims

    for v, var in ds.variables.items():
        variable = EOVariableBuilder(v, default_attrs = False)
        variable.dtype = var.dtype
        variable.dimensions = var.dims
        variable.attrs = var.attrs
        group.variables.append(variable)

    product.groups.append(group)

product.attrs['metadata_files'] = '[xfdumanfist.xml]'
print("inputs read")

IndexError: list index out of range

In [6]:
product.compute()